# Required libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ee # install with : pip install earthengine-api
import geemap
import geetools
import geopandas as gpd
import pandas as pd
from geopandas import GeoDataFrame
from shapely.geometry import Point,Polygon
import folium
import json

# Sentinel-2 Cloud Masking with s2cloudless

The first part of this script describes a method for downloading NDWI images, not by selecting a single satellite image from the S2 database at a specific point in time, but rather by starting with a library of images taken between two dates and combining them. This approach addresses the issue of cloud cover. It allows for handling large volumes of data without the need to create mosaics or search for the perfect image, if such an image even exists.  

This method builds on several existing works available at the following links:  

https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless

https://medium.com/sentinel-hub/improving-cloud-detection-with-machine-learning-c09dc5d7cf13

https://developers.google.com/earth-engine/api_docs

# Script 

## Authentification to GEE

In [2]:
# Trigger the authentication flow.
ee.Authenticate(auth_mode='notebook')

True

In [3]:
# Initialize the library.
ee.Initialize()


## Import Area of interest (AOI) as polygon 

In [6]:
# Should be in geojson format and ESPG 4326
shape = gpd.read_file(r"AOI_example_3413.geojson")
js = json.loads(shape.to_json())
geometry = ee.Geometry(ee.FeatureCollection(js).geometry())

## Variable and function definitions

In [7]:
# Variables
AOI = geometry
START_DATE = '2016-08-14'
END_DATE = '2020-08-15'
CLOUD_FILTER = 10
CLD_PRB_THRESH = 40
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 2
BUFFER = 100

<ul>
<li>AOI	 =	Area of interest
<li>START_DATE	 =	Image collection start date (inclusive)
<li>END_DATE	=	Image collection end date (exclusive)
<li>CLOUD_FILTER	=	Maximum image cloud cover percent allowed in image collection
<li>CLD_PRB_THRESH	 =	Cloud probability (%); values greater than are considered cloud
<li>NIR_DRK_THRESH	 =	Near-infrared reflectance; values less than are considered potential cloud shadow
<li>CLD_PRJ_DIST	=	Maximum distance (km) to search for cloud shadows from cloud edges
<li>BUFFER	 =	Distance (m) to dilate the edge of cloud-identified objects
<ul>

In [8]:
# Functions

def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))


################################################################################################################L


def import_and_join(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))


################################################################################################################


def add_cloud_bands(img):
    # Get s2cloudless image, select the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))


################################################################################################################


def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))


################################################################################################################


def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/10)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 10})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)


################################################################################################################


# Define a method for displaying Earth Engine image tiles to a folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)
    
    
################################################################################################################


def display_cloud_layers(col):
    # Mosaic the image collection.
    img = col.mosaic()

    # Subset layers and prepare them for display.
    clouds = img.select('clouds').selfMask()
    shadows = img.select('shadows').selfMask()
    dark_pixels = img.select('dark_pixels').selfMask()
    probability = img.select('probability')
    cloudmask = img.select('cloudmask').selfMask()
    cloud_transform = img.select('cloud_transform')

    # Create a folium map object.
    center = AOI.centroid(10).coordinates().reverse().getInfo()
    m = folium.Map(location=center, zoom_start=12)

    # Add layers to the folium map.
    m.add_ee_layer(img,
                   {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                   'S2 image', True, 1, 9)
    m.add_ee_layer(probability,
                   {'min': 0, 'max': 100},
                   'probability (cloud)', False, 1, 9)
    m.add_ee_layer(clouds,
                   {'palette': 'e056fd'},
                   'clouds', False, 1, 9)
    m.add_ee_layer(cloud_transform,
                   {'min': 0, 'max': 1, 'palette': ['white', 'black']},
                   'cloud_transform', False, 1, 9)
    m.add_ee_layer(dark_pixels,
                   {'palette': 'orange'},
                   'dark_pixels', False, 1, 9)
    m.add_ee_layer(shadows, {'palette': 'yellow'},
                   'shadows', False, 1, 9)
    m.add_ee_layer(cloudmask, {'palette': 'orange'},
                   'cloudmask', True, 0.5, 9)

    # Add a layer control panel to the map.
    m.add_child(folium.LayerControl())

    # Display the map.
    display(m)

    
################################################################################################################
    

def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # select reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

## Import and filter Sentinel2 data

In [9]:
S2_collection = import_and_join(AOI, START_DATE, END_DATE)
S2_collection_months_filter = S2_collection.filter(ee.Filter.calendarRange(6, 8, 'month'))

C:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\ee\deprecation.py:207: DeprecationWarning: 

Attention required for COPERNICUS/S2_SR! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR

  warnings.warn(warning, category=DeprecationWarning)


## Vizualisation of cloud mask and NDWI

### Vizualisation initialisation

In [10]:
# Add the Earth Engine layer method to folium.
folium.Map.add_ee_layer = add_ee_layer

### Vizualisation of cloud mask 

In [11]:
S2_collection_months_filter_with_cloud_components = S2_collection_months_filter.map(add_cld_shdw_mask)

display_cloud_layers(S2_collection_months_filter_with_cloud_components)

### Vizualisation of cloudless mosaic image 

In [12]:
S2_mosaic = S2_collection_months_filter_with_cloud_components.map(apply_cld_shdw_mask).mosaic()
S2_mean = S2_collection_months_filter_with_cloud_components.map(apply_cld_shdw_mask).mean()
s2_median = S2_collection_months_filter_with_cloud_components.map(apply_cld_shdw_mask).median()

In [13]:
# Create a folium map object.
center = AOI.centroid(10).coordinates().reverse().getInfo()
m = folium.Map(location=center, zoom_start=12)

# Add layers to the folium map.
m.add_ee_layer(S2_mosaic,
                {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1},
                'S2 cloud-free mosaic')

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)

### NDWI calculation and visualisation

In [26]:
ndwi = S2_mosaic.normalizedDifference(['B3', 'B8']).rename('NDWI')


m1 = folium.Map(location=center, zoom_start=12)

# Add layers to the folium map.
m1.add_ee_layer(ndwi.clip(AOI), {'bands': ['NDWI'], 'min': -1, 'max': 1 }, 'NDWI')

# Add a layer control panel to the map.
m1.add_child(folium.LayerControl())

# Display the map.
display(m1)

## Image Download

In [15]:
# Clip to AOI before download to limit computation time

ndwi_clipped = ndwi.clip(AOI)



In [21]:
# Task creation, be careful to set-up file name, folder to export to on google drive and scale 
#(scale might be inacurate, always check it after downolad on GIS and manually correct if necessary)

task_ndwi = ee.batch.Export.image.toDrive(ndwi_clipped, 'NDWI_file_name', folder='export', region=AOI, scale=10, crs=None, maxPixels = 10000000000000 )


In [22]:
# Task start-up

task_ndwi.start()


In [25]:
# Basic task check-up

task_ndwi.status()



{'state': 'RUNNING',
 'description': 'NDWI_file_name',
 'priority': 100,
 'creation_timestamp_ms': 1735419506077,
 'update_timestamp_ms': 1735419544446,
 'start_timestamp_ms': 1735419515565,
 'task_type': 'EXPORT_IMAGE',
 'attempt': 1,
 'id': 'QWV3LGLFQ2MKN4U4NCOGHA53',
 'name': 'projects/508923293188/operations/QWV3LGLFQ2MKN4U4NCOGHA53'}

### Go to this link to monitor the task in real time :

### https://code.earthengine.google.com/tasks

